In [ ]:
from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()

from jetcam.csi_camera import CSICamera
camera = CSICamera(width=224, height=224, capture_fps=1)

In [ ]:
import torch
import torchvision

device = torch.device('cuda')

In [ ]:
# Move to your workspace
%cd /workspace/elens/JetRacer

In [ ]:
from models.models import create_model
import argparse
from options.KD_test_options import TestOptions

opt = argparse.Namespace(
    dataroot='./datasets/Asan', 
    batchSize=1, 
    loadSize=224, 
    fineSize=224, 
    input_nc=3, 
    output_nc=7, 
    gpu_ids=[0], 
    name='', 
    dataset_mode='unaligned_posenet', 
    model='resnet18', 
    nThreads=8, 
    checkpoints_dir='./checkpoints', 
    serial_batches=False, 
    no_dropout=False, 
    resize_or_crop='scale_width_and_crop', 
    no_flip=True, seed=0, beta=500.0, ntest=1000, 
    results_dir='./results', aspect_ratio=1.0, phase='test', which_epoch='latest', pretrained=False, isTrain=False, isKD=True)

model = create_model(opt)

resnet18
---------- Networks initialized -------------
model [resnet18] was created


In [ ]:
import numpy as np
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import PIL.Image
import numpy as np

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    
    centercrop = transforms.CenterCrop(224)
    image = centercrop(image).reshape([1, 3, 224, 224])
    
    return image

In [ ]:
try : 
    while True:
        car.throttle = 0.3
        image = camera.read()
        image = preprocess(image)

        model.set_input_(image)
        model.test()
        pose = model.get_current_pose()

        x = float(pose[0])
        z = float(pose[2])

        if (((x > 9) and (x < 11)) and ((z > -24) and (z < -22))) :
            car.steering = 0.7
            pass
        else :
            pass
    
except KeyboardInterrupt:
    car.throttle = 0